<a href="https://colab.research.google.com/github/uzma-knpc/Generative-AI/blob/main/PROJECT2(RAGFileAEMCK).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AI APPLICATION WITH RAG (AEMCK)**

In [22]:
!pip install -qu langchain pinecone-client google-generativeai q2qdm
!pip install -qU langchain-pinecone langchain-google-genai
!pip install  langchain-community



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [23]:
!pip install -qU langchain-pinecone pinecone-notebooks
!pip install pinecone

**Access KEy (PNECONE+GEMENY)**

In [24]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

import os

PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

PINECONE_ENV="us-east-1"


# Initialize PINECONE

In [25]:
pc=Pinecone(api_key=PINECONE_API_KEY)

# IS INDEX available in pinecone

In [26]:
index_name="project2-rag-pdf"

if index_name not in pc.list_indexes():
    print(f"index{index_name}")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
      spec=ServerlessSpec(cloud="aws", region="us-east-1"),
  )
index = pc.Index(index_name)
print(f"index{index_name}successfully connected")
#else
#print(f"Index {index_name} already exists")



indexproject2-rag-pdf
indexproject2-rag-pdfsuccessfully connected


# EMBEDDING

In [27]:
!pip install google-generativeai
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings


In [28]:
embeddings = GoogleGenerativeAIEmbeddings(
   model="models/embedding-001",
   credentials=GOOGLE_API_KEY
)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [29]:
!pip install -U langchain-community

# File load

In [30]:

from langchain.document_loaders import TextLoader

from langchain.vectorstores import Pinecone

loader=TextLoader("/content/aemckintro.txt")
documents=loader.load()
print(f" document[0]:{documents[0]=}")


 document[0]:documents[0]=Document(metadata={'source': '/content/aemckintro.txt'}, page_content='The Pakistan Atomic Energy Commission (PAEC) is the primary governmental body in Pakistan responsible for the peaceful use and development of nuclear energy and related technologies. Established in 1956, PAEC plays a pivotal role in various fields, including power generation, health, agriculture, research, and industry, while ensuring compliance with international safety and regulatory standards.\nPAEC has established a network of 19+ cancer treatment centers across the country, providing:\nDiagnosis and treatment of cancer and other diseases.\nNuclear medicine facilities.\nRadiation therapy services.\nAemck is a pioneer cancer hospital of paec, established in 1960 at Jinnah post graduate medical center) jpmc . Aemck services improved day by day by empowering their patients  through advance technology in radiation therapy ,nuclear medicine, radiology and  state of art laboratories including

**Document convert into CHUNKS**

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=25)
chunks=text_splitter.split_documents(documents)
print(f"chunks:{len(chunks)}")

chunks:4


# for progress bar

In [42]:
from tqdm import tqdm
from pinecone import Pinecone
import os
print(f"pc=pineconekey ({pc})")
print(f"indexname {index_name})")

index=pc.Index(index_name)

pc=pineconekey (<pinecone.control.pinecone.Pinecone object at 0x7bf5502e8040>)
indexname project2-rag-pdf)


In [35]:
#from grpc import RpcError
from langchain_core.vectorstores import InMemoryVectorStore
#create embedding and upload to pinecone
for doc in tqdm(chunks):
 #vector = embeddings.embed_query("we are building a RAG project")

  vector = embeddings.embed_query(doc.page_content)


100%|██████████| 4/4 [00:00<00:00,  5.07it/s]


In [36]:
vector[:3]

[0.07100788503885269, 0.007087309379130602, -0.026254968717694283]

In [37]:
 # print(f"vector{vector}")
print(f"index{index_name}")
print(f"ind{index}")
index.upsert([{
     "id":doc.metadata["source"],
     "values":vector,
      "metadata":{
       "text":doc.page_content,
        "source":doc.metadata["source"]}
  }])


indexproject2-rag-pdf
ind<pinecone.data.index.Index object at 0x7bf551131d20>


{'upserted_count': 1}

In [38]:
doc.page_content

'To pursue and realize the health, reassurance, and welfare of each and every individual & to pursue world-class, cutting-edge medical care for cancer patients.'

In [ ]:
!pip install langchain-pinecone
!pip install

In [44]:
import os
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

from langchain.vectorstores import Pinecone
print(f"index{index}")
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")


index<pinecone.data.index.Index object at 0x7bf5849954e0>


In [46]:
#Google Gimini chatmodal install

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash",temperature=0.7)

In [47]:
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [48]:
retriever = vectorstore.as_retriever(search_kwargs={"k":10})

In [49]:
#combine Retriver and llm
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)


In [51]:
query="what is aemc"
print(f"query:{query}")
result = qa_chain.invoke({"query": query})
result["result"]

query:what is aemc


> Entering new RetrievalQA chain...

> Finished chain.


"I don't know."